In [1]:
import collections
import os
import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Embedding, Dropout, TimeDistributed
from keras.layers import LSTM, CuDNNLSTM
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
import numpy as np
import pandas as pd
from keras.layers import Lambda
from keras.models import Model
from keras.layers import Input
import keras
import gensim
from gensim.models import Word2Vec

/Users/francisdamachi/anaconda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
/Users/francisdamachi/anaconda/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Version with timedistributed

In [5]:

def generate(model, start= 1, length_generate = 7) :
    print(reversed_dictionary[start])
    start_word = start
    built_phrase = [start_word]

    seed_text = np.array([start_word])
    seed_text = pad_sequences([seed_text],maxlen=7, padding='post')
    predictions = model.predict_classes(seed_text, verbose=0)


    for i in range(length_generate): 
      predict = predictions[0][i]
      built_phrase.append(predict)
      seed_text = pad_sequences([built_phrase], maxlen=7, padding="post")
      predictions = model.predict_classes(seed_text, verbose =0)

    return built_phrase
def read_words(filename):
    with tf.gfile.GFile(filename, "rb") as f:
        return f.read().decode("utf-8").replace("\n", "eos").split()
      
def build_vocab(filename):
    data = read_words(filename)

    counter = collections.Counter(data)
    count_pairs = sorted(counter.items(), key=lambda x: (-x[1], x[0]))

    words, _ = list(zip(*count_pairs))
    word_to_id = dict(zip(words, range(1,len(words)+1)))

    return word_to_id
  
  
def file_to_word_ids(filename, word_to_id):
    data = read_words(filename)
    return [word_to_id[word] for word in data if word in word_to_id]
  
  
def load_data(file):
    # get the data paths
    train_path = os.path.join(data_path,file)
    # build the complete vocabulary, then convert text data to list of integers
    word_to_id = build_vocab(train_path)
    train_data = file_to_word_ids(train_path, word_to_id)
    
    vocabulary = len(word_to_id)
    reversed_dictionary = dict(zip(word_to_id.values(), word_to_id.keys()))

    print(train_data[:5])
    print(word_to_id)
    print(vocabulary)
    print(" ".join([reversed_dictionary[x] for x in train_data[:10]]))
    return train_data, vocabulary, reversed_dictionary
  
  
class KerasBatchGenerator(object):

    def __init__(self, data, num_steps, batch_size, vocabulary, skip_step):
        self.data = data
        self.num_steps = num_steps
        self.batch_size = batch_size
        self.vocabulary = vocabulary+1
        # this will track the progress of the batches sequentially through the
        # data set - once the data reaches the end of the data set it will reset
        # back to zero
        self.current_idx = 0
        # skip_step is the number of words which will be skipped before the next
        # batch is skimmed from the data set
        self.skip_step = skip_step


    def generate(self):

        x = np.zeros((self.batch_size, self.num_steps))
        y = np.zeros((self.batch_size, self.num_steps, self.vocabulary))
        while True:
            for i in range(self.batch_size):
                if self.current_idx + self.num_steps >= len(self.data):
                    # reset the index back to the start of the data set
                    self.current_idx = 0
                data_tmp = self.data[self.current_idx:self.current_idx + self.num_steps]
                x[i, :] = data_tmp
                temp_y = self.data[self.current_idx + 1:self.current_idx + self.num_steps + 1]
                # convert all of temp_y into a one hot representation
                y[i, :, :] = to_categorical(temp_y, num_classes=self.vocabulary)
                
                self.current_idx += self.skip_step
            yield x, y


In [122]:
data_path = './'
num_steps = 7
batch_size = 1
skip_step = num_steps + 1
hidden_size = 50
num_epochs = 500
input_shape = num_steps


train_data, vocabulary, reversed_dictionary = load_data('data.txt')
train_data_generator = KerasBatchGenerator(train_data, num_steps, batch_size, vocabulary,
                                           skip_step)



[4, 2, 5, 23, 21]
{'pail': 17, 'jill': 5, 'jack': 4, 'went': 23, 'fell': 12, 'of': 16, 'came': 10, 'down': 3, 'broke': 9, 'the': 18, 'hill': 14, 'to': 19, 'water': 22, 'fetch': 13, 'up': 21, 'after': 7, 'crown': 11, 'tumbling': 20, 'his': 15, 'big': 8, 'eos': 1, 'a': 6, 'and': 2}
23
jack and jill went up the hill eos to fetch


In [1]:


model = Sequential()
model.add(Embedding(vocabulary+1, 10, input_length=num_steps))
model.add(LSTM(hidden_size, return_sequences=True))
model.add(LSTM(hidden_size, return_sequences=True))
model.add(TimeDistributed(Dense(vocabulary+1)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
model.fit_generator(train_data_generator.generate(),len(train_data)//(batch_size* num_steps),epochs=num_epochs)



NameError: name 'Sequential' is not defined

In [124]:
word2id  = dict([(v,k) for k,v in reversed_dictionary.items()])

In [125]:
#text = generate(model,20)

# = sad, anger, anx, negemo, posemo

import keras

def generate(model, start= 'jack', length_generate = 7) :
    
    start_ = start.split(" ")
    start = [word2id[w] for w in start_]
    start_word = start
    built_phrase = start_word

    seed_text = np.array(start_word)
    seed_text = keras.preprocessing.sequence.pad_sequences([seed_text],maxlen=7, padding='post')
    

    predictions = model.predict_classes(seed_text, verbose=0)
    
    for i in range(length_generate): 
      predict = predictions[0][i]
      built_phrase.append(predict)
     
      seed_text = keras.preprocessing.sequence.pad_sequences([built_phrase], maxlen=length_generate, padding="post")
      predictions = model.predict_classes(seed_text, verbose =0)

    return [reversed_dictionary[w] for w in built_phrase]


In [140]:
generate(model,'jack up')

['jack', 'up', 'and', 'jill', 'went', 'up', 'the', 'hill', 'eos']